In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [ ]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
from langchain_openai import ChatOpenAI


In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


In [ ]:


class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
message = {
    "role": "user",
    "content": "Who walked on the moon for the first time? Print only the name."
}

response = graph.invoke({"messages": [message]})
print(response["messages"][-1].content)


In [ ]:
state = None

while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break

    if state is None:
        state = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append(
            {"role": "user", "content": in_message}
        )

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)


In [ ]:
import gradio as gr

def chat(user_input, history):
    result = graph.invoke({
        "messages": [
            {"role": "user", "content": user_input}
        ]
    })
    return result["messages"][-1].content


gr.ChatInterface(
    fn=chat,
    title="Simple LangGraph Chatbot",
    description="LangGraph chatbot running inside a notebook"
).launch(inline=True)
